In [1]:
from pyarrow import csv

In [2]:
read_options = csv.ReadOptions(
               column_names=["time","source user@domain","destination user@domain", \
                             "source computer","destination computer","authentication type", \
                             "logon type","authentication orientation","success/failure","day"])


In [ ]:
table_file = csv.read_csv('../lm-vol/LANL_train.csv', read_options=read_options)

In [ ]:
help(table_file.filter)

In [ ]:
pandas_lanl = table_file.to_pandas()

In [ ]:
pandas_lanl.shape

In [ ]:
pandas_lanl.columns

In [10]:
import pandas as pd 
import torch
from torch import Tensor

In [11]:
pandas_lanl.sample(5)

,time,source user@domain,destination user@domain,source computer,destination computer,authentication type,logon type,authentication orientation,success/failure,day
171,170.0,760075,U1653@DOM1,C17693,C395,U1653@DOM1,NTLM,Network,LogOn,Success
246,245.0,767813,U66@DOM1,C17693,C2944,U66@DOM1,NTLM,Network,LogOn,Success
415,414.0,1076742,U453@DOM1,C17693,C2388,U453@DOM1,NTLM,Network,LogOn,Success
97,96.0,739706,U1600@DOM1,C17693,C92,U1600@DOM1,NTLM,Network,LogOn,Success
250,249.0,768546,U4353@DOM1,C17693,C17860,U4353@DOM1,NTLM,Network,LogOn,Success


In [10]:
pandas_lanl = pandas_lanl.rename(columns={'source user@domain': "src_user", 'destination user@domain': "dest_user"})

In [11]:
df = pandas_lanl ##shallow copy lol

In [12]:
df.columns

Index(['time', 'src_user', 'dest_user', 'source computer',
       'destination computer', 'authentication type', 'logon type',
       'authentication orientation', 'success/failure', 'day'],
      dtype='object')

In [13]:
df = df[(df['src_user'].str.contains('LOCAL')==False) & (df['src_user'].str.contains('SYSTEM')==False) & 
        (df['src_user'].str.contains('ANONYMOUS')==False) & (df['src_user'].str.contains('ADMINISTRARTOR')==False) &
        (df['src_user'].str.contains('$')==False) 
       ]

df = df[(df['dest_user'].str.contains('LOCAL')==False) & (df['dest_user'].str.contains('SYSTEM')==False) & 
        (df['dest_user'].str.contains('ANONYMOUS')==False) & (df['dest_user'].str.contains('ADMINISTRARTOR')==False) &
        (df['dest_user'].str.contains('$')==False) 
       ]


In [14]:
df.shape

(0, 10)

In [16]:
pandas_lanl.sample(2)

,time,src_user,dest_user,source computer,destination computer,authentication type,logon type,authentication orientation,success/failure,day
620080501,4521322,U341@DOM1,U341@DOM1,C612,C612,?,Network,LogOff,Success,53
99001974,1490866,C2070$@DOM1,C2070$@DOM1,C2071,C2071,?,?,TGT,Success,18


In [1]:
####decide to play with unique test data 
import pandas as pd 
import torch
from torch import Tensor

In [2]:

pandas_lanl = pd.read_csv("../lm-vol/redteam_complete.csv")

In [3]:
pandas_lanl.shape

(739, 10)

In [4]:
pandas_lanl.columns

Index(['Unnamed: 0', 'time', 'source', 'source_computer',
       'destination_computer', 'destination', 'authentication_type',
       'logon_type', 'authentication_orientation', 'Success'],
      dtype='object')

In [6]:
client_service = pandas_lanl[['source','destination_computer']].copy()
client_service.sample(5)

,source,destination_computer
112,U5087@DOM1,C353
81,U349@DOM1,C3601
689,U3005@DOM1,C2012
357,U1289@DOM1,C2519
548,U66@DOM1,C3610


In [7]:
#df.rename(columns={"A": "a", "B": "c"})
client_service = client_service.rename(columns={"source":"client","destination_computer":"service"})

In [8]:
unique_client_id = client_service['client'].unique()
unique_client_id = pd.DataFrame(data={
     'clientId': unique_client_id,
     'mappedID': pd.RangeIndex(len(unique_client_id)),
 })

print(unique_client_id)



unique_service_id = client_service['service'].unique()
unique_service_id = pd.DataFrame(data={
     'serviceId': unique_service_id,
     'mappedID': pd.RangeIndex(len(unique_service_id)),
 })

        clientId  mappedID
0      U620@DOM1         0
1      U748@DOM1         1
2     U6115@DOM1         2
3      U636@DOM1         3
4     U1723@DOM1         4
..           ...       ...
99    U8840@DOM1        99
100   U1306@DOM1       100
101   U3764@DOM1       101
102  U1467@C3597       102
103   U3406@DOM1       103

[104 rows x 2 columns]


In [9]:
print(len(unique_client_id), len(unique_service_id))

104 295


In [10]:
client_service

,client,service
0,U620@DOM1,C1003
1,U748@DOM1,C305
2,U748@DOM1,C728
3,U6115@DOM1,C1173
4,U636@DOM1,C294
...,...,...
734,U655@DOM1,C828
735,U655@DOM1,C828
736,U655@DOM1,C828
737,U655@DOM1,C828


In [11]:
# Perform merge to obtain the edges from users and movies:
client_id = pd.merge(client_service['client'], unique_client_id, left_on='client', right_on='clientId', how='left')
client_id = torch.from_numpy(client_id['mappedID'].values)
service_id = pd.merge(client_service['service'], unique_service_id, left_on='service', right_on='serviceId', how='left')
service_id = torch.from_numpy(service_id['mappedID'].values)

# following PyG semantics:
edge_index_client_to_service = torch.stack([client_id, service_id], dim=0)

print(edge_index_client_to_service)


tensor([[  0,   1,   1,  ...,  82,  82,   5],
        [  0,   1,   2,  ..., 273, 273, 294]])


In [16]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T


In [17]:
# pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

In [12]:
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

data = HeteroData()
data["client"].node_id = torch.arange(len(unique_client_id))
data["service"].node_id = torch.arange(len(unique_service_id))

data["client", "auth_attempt", "service"].edge_index = edge_index_client_to_service

Transform = T.ToUndirected()
data = Transform(data)

In [13]:
type(data)

torch_geometric.data.hetero_data.HeteroData

In [17]:
from torch_geometric.loader import LinkNeighborLoader
edge_label_index = val_data["client", "auth_attempt", "service"].edge_label_index
edge_label = val_data["client", "auth_attempt", "service"].edge_label

val_loader = LinkNeighborLoader(
    data=data,
    num_neighbors=[20, 10],
    edge_label_index=(("client", "auth_attempt", "service"), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

In [18]:
print("Sampled mini-batch:")
print("===================")
print(sampled_data)

Sampled mini-batch:
HeteroData(
  client={ node_id=[100] },
  service={ node_id=[258] },
  (client, auth_attempt, service)={
    edge_index=[2, 671],
    input_id=[219],
    edge_label_index=[2, 219],
    edge_label=[219]
  },
  (service, rev_auth_attempt, client)={ edge_index=[2, 567] }
)


In [ ]:
from sklearn.metrics import roc_auc_score

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        # TODO: Collect predictions and ground-truths and write them into
        # `preds` and `ground_truths`.
        sampled_data = sampled_data.to(device)
        preds.append(model.forward(sampled_data))
        ground_truths.append(sampled_data["client", "auth_attempt", "service"].edge_label)
        
        # raise NotImplementedError

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()
auc = roc_auc_score(ground_truth, pred)
print()
print(f"Validation AUC on RED TEAM: {auc:.4f}")